<a href="https://colab.research.google.com/github/calvinjohnson747/hw2/blob/main/CS678_HW2_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2, CS678 Fall 2023

### This is due on October 13th, 2023.

Submit the report to Gradescope with the naming convention of
**John_Doe_HW2_Report_CS678_F23.pdf** if your name is John Doe.
Only the report should be submitted to Gradescope. The rest of the code and data files,
including this notebook in its completion form,
should be submitted to Blackboard in a single zipped folder.

<!-- #### IMPORTANT:

After copying this notebook to your Google Drive, please paste a link to it
below. To get a publicly-accessible link, hit the *Share* button at the top
right, then click "Get shareable link" and copy over the result.If you fail to
do this, you will receive no credit for this homework!

***LINK:***

--- -->


##### *How to submit this problem set:*
- Write all the answers in this notebook.

- When creating your final version of the notebook to hand in,
  please do a fresh restart and execute every cell in order.
  One handy way to do this is by clicking `Runtime -> Run All` in the notebook menu.

##### *Policy regarding Google Colab:*
- The instruction in this notebook assumes that you will use Colab.

- However, using Colab is not required. You are free to run the code on your local machine, though in that case you may suffer from a slow runtime due to the lack of proper GPU resources.

---

##### *Academic honesty*

- We will audit the notebooks from a set number of students, chosen at
  random. The audits will check that the code you wrote actually generates the
  answers in your report PDF. If you turn in correct answers on your PDF without code
  that actually generates those answers, we will consider this a serious case of
  cheating. See the course page for honesty policies.

- We will also run automatic checks of notebooks for plagiarism.
  Copying code from others is also considered a serious case of cheating.

---

# Part 1: Data Collection and Annotation

In this homework, you will first collect a labeled dataset of **150** sentences for a text classification task of your choice. This process will include:

1. *Data collection*: Collect 150 sentences from any source you find interesting (e.g., literature, Tweets, news articles, reviews, etc.)

2. *Task design*: Come up with a multilabel sentence-level classification task that you would like to perform on your sentences.

3. On your dataset, collect annotations from **two** classmates for your task on a **second, separate set** of a minimum of **150** sentences. Everyone in this class will need to both create their own dataset and also serve as an annotator for two other classmates. In order to get everything done on time, you need to complete the following steps:

> *   Find two classmates willing to label 150 sentences each (use the Piazza "search for teammates" thread if you're having issues finding labelers).
*   Collect the labeled data from each of the two annotators.
*   Sanity check the data for basic cleanliness (are all examples annotated? are all labels allowable ones?)

4. Collect feedback from annotators about the task including annotation time and obstacles encountered (e.g., maybe some sentences were particularly hard to annotate!)

5. Calculate and report inter-annotator agreement.

6. Aggregate output from both annotators to create final dataset (include your first 150 sentences too).

7. Perform NLP experiments on your new dataset!

The mapping of label names and IDs in seed.tsv is as follows:

```json
{
    'Economic': 1.0,
    'Capacity and Resources': 2.0,
    'Morality': 3.0,
    'Fairness and Equality': 4.0,
    'Legality, Constitutionality, Jurisdiction': 5.0,
    'Policy Prescription and Evaluation': 6.0,
    'Crime and Punishment': 7.0,
    'Security and Defense': 8.0,
    'Health and Safety': 9.0,
    'Quality of Life': 10.0,
    'Cultural Identity': 11.0,
    'Public Sentiment': 12.0,
    'Political': 13.0,
    'External Regulation and Reputation': 14.0,
    'Other': 15.0
}
```

Make sure that this mapping is followed in all of your data files.

## Question 3 (8 points):
Now, compute the inter-annotator agreement between your two annotators. Upload both .tsv files to your Colab session (click the folder icon in the sidebar to the left of the screen). In the code cell below, read the data from the two files and compute both the raw agreement (% of examples for which both annotators agreed on the label) and the [Cohen's Kappa](https://en.wikipedia.org/wiki/Cohen%27s_kappa). Feel free to use implementations in existing libraries (e.g., [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html)). After you’re done, report the raw agreement and Cohen’s scores in your report.

*If you're curious, Cohen suggested the Kappa result be interpreted as follows: values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight, 0.21–0.40 as fair, 0.41– 0.60 as moderate, 0.61–0.80 as substantial, and 0.81–1.00 as almost perfect agreement.*

In [ ]:
### WRITE CODE TO LOAD ANNOTATIONS AND
### COMPUTE AGREEMENT + COHEN'S KAPPA HERE!
import pandas as pd
from sklearn.metrics import cohen_kappa_score

def raw_agreement(annotator1_df, annotator2_df):
    agreed_count = (annotator1_df['label'] == annotator2_df['label']).sum()
    total = len(annotator1_df)
    raw_agreement = (agreed_count/total)*100
    return raw_agreement

def kohens_cappa(annotator1_df, annotator2_df):
    # TODO: Implement this function
    cks = cohen_kappa_score(annotator1_df['label'], annotator2_df['label'])
    return cks

# TODO: load your data correctly
annotator1_df = pd.read_csv('/annotator1.tsv',sep='\t')
annotator2_df = pd.read_csv('/annotator2.tsv',sep='\t')

print("--- Raw agreement between annotator1 and annotator2 ---")
print(raw_agreement(annotator1_df, annotator2_df))

print("--- Cohen's kappa score between annotator1 and annotator2 ---")
print(kohens_cappa(annotator1_df, annotator2_df))

TODO : Write the values obtained above in this cell.

### *RAW AGREEMENT*: __42.0____

### *COHEN'S KAPPA*: ___37.18____

# Part 2: Model Training and Testing

Now we'll move onto fine-tuning  pretrained language models specifically on your dataset. This part of the homework is meant to be an introduction to the HuggingFace library, and it contains code that will potentially be useful for your final projects. Since we're dealing with large models, the first step is to change to a GPU runtime.

## Adding a hardware accelerator

Please go to the menu and add a GPU as follows:

`Edit > Notebook Settings > Hardware accelerator > (GPU)`

Run the following cell to confirm that the GPU is detected.

In [ ]:
import torch
torch.cuda.empty_cache()

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [ ]:
import random
import numpy as np

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_everything()

## Installing Hugging Face's Transformers library
We will use Hugging Face's Transformers (https://github.com/huggingface/transformers), an open-source library that provides general-purpose architectures for natural language understanding and generation with a collection of various pretrained models made by the NLP community. This library will allow us to easily use pretrained models like `BERT` and perform experiments on top of them. We can use these models to solve downstream target tasks, such as text classification, question answering, and sequence labeling.

Run the following cell to install Hugging Face's Transformers library and download a sample data file called seed.tsv that contains 250 sentences in English, annotated with their frame.

In [ ]:
!pip install transformers
!pip install -U -q PyDrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00


The cell below imports some helper functions we wrote to demonstrate the task on
the sample seed dataset.

#### *IMPORTANT NOTE*:

The tokenize_and_format function in helpers.py uses bert-base-uncased as the
model for the tokenizer. If you are using a different model for training in this
notebook or for running predictions in a different notebook or python file, you
need to change the model name as well in the tokenizer, otherwise you will get
arbitrarily incorrect results down the line.

If you update the model name for the tokenizer, you would need to reload the
file which can be done simply by re-running the cell below.

In [ ]:
from helpers import tokenize_and_format, flat_accuracy

# Part 1: Data Prep and Model Specifications

Upload your data using the file explorer to the left. We have provided a
function below to tokenize and format your data as BERT requires. Make sure that
your tsv file, titled final_data.tsv, has one column "sentence" and another
column "label_ID" containing integers/float. (basically the same format as
seed.tsv should be maintained for the sentence and label columns)

If you run the cell below without modifications, it will run on the seed.tsv
example data we have provided. It imports some helper functions we wrote to
demonstrate the task on the sample dataset. You should first run all of the
following cells with seed.tsv just to see how everything works. Then, once you
understand the whole preprocessing / fine-tuning process, change the tsv in the
below cell to your final_data.tsv file, add any extra preprocessing code you
wish, and then run the cells again on your own data.


#### Important Note :

The code below expects the data to be in a tsv file  with the columns as "sentence"
and "label_ID" (other columns are not that relevant here). But this is different
from the instructions in the report where you are expected to create data with
"text" and "label" columns for all of the annotation steps.

Modify the code below to suitably handle this.

In [ ]:
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
import numpy as np


seed_everything()

df = pd.read_csv('final_data_english.tsv', sep='\t') # TODO : Uncomment this line to use the full dataset
#df = pd.read_csv('seed.tsv', sep='\t')

df = df.sample(frac=1).reset_index(drop=True)

df.to_csv('final_data.tsv', sep='\t')

index = df.index.values #this saves index
texts = df.text.values # this assumes that the column containing the text is called "sentence"
label = df.label.values # this assumes that the column containing the labels is called "label_ID"


### tokenize_and_format() is a helper function provided in helpers.py ###
### Make sure you use the correct model name for your tokenizer! ###
input_ids, attention_masks = tokenize_and_format(texts)

label_list = []
for l in label:
  label_array = np.zeros(len(set(label)))
  label_array[int(l)-1] = 1
  label_list.append(label_array)

print(set(label))
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(np.array(label_list))

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])
print('Index:', index[0])



{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
Original:  Immigration policies must align with our constitutional principles, safeguarding rights for all. Let's advocate for fairness and legality.
Token IDs: tensor([  101,  7521,  6043,  2442, 25705,  2007,  2256,  6543,  6481,  1010,
        28805,  2075,  2916,  2005,  2035,  1012,  2292,  1005,  1055,  8175,
         2005, 26935,  1998,  3423,  3012,  1012,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
Index: 0


In [ ]:
#Augmented Data
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
import numpy as np


seed_everything()

df_a = pd.read_csv('augmented_data.tsv', sep='\t') # TODO : Uncomment this line to use the full dataset
#df = pd.read_csv('seed.tsv', sep='\t')

df_a = df_a.sample(frac=1).reset_index(drop=True)


index_a = df_a.index.values #this saves index
texts_a = df_a.text.values # this assumes that the column containing the text is called "sentence"
label_a = df_a.label_id.values # this assumes that the column containing the labels is called "label_ID"


### tokenize_and_format() is a helper function provided in helpers.py ###
### Make sure you use the correct model name for your tokenizer! ###
input_ids_a, attention_masks_a = tokenize_and_format(texts_a)

label_list_a = []
for l in label_a:
  label_array = np.zeros(len(set(label_a)))
  label_array[int(l)-1] = 1
  label_list_a.append(label_array)

print(set(label_a))
# Convert the lists into tensors.
input_ids_a = torch.cat(input_ids_a, dim=0)
attention_masks_a = torch.cat(attention_masks_a, dim=0)
labels_a = torch.tensor(np.array(label_list_a))

# Print sentence 0, now as a list of IDs.
print('Original: ', texts_a[0])
print('Token IDs:', input_ids_a[0])
print('Index:', index_a[0])



{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
Original:  Kuhalalisha ndoa ya jinsia moja huwapa wanandoa fursa ya kuanzisha rasmi kujitolea wao kwa kila mmoja na kuonyesha wazi upendo wao.
Token IDs: tensor([  101, 13970, 19531, 13602,  2050,  1050,  3527,  2050,  8038,  9743,
         8464,  9587,  3900, 15876,  4213,  4502, 14071, 28574,  2050,  6519,
         3736,  8038, 13970,  2319,  5831,  7377, 20710,  4328, 13970, 18902,
         9890,  2050, 11333,  2080,  6448,  2050, 11382,  2721,  3461, 22918,
         6583, 13970, 16585,  9953,  2050, 11333,  5831,  2039, 10497,  2080,
        11333,  2080,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
Index: 0


## Create train/test/validation splits

Here we split your dataset into 3 parts: a training set, a validation set, and a testing set. Each item in your dataset will be a 3-tuple containing an input_id tensor, an attention_mask tensor, and a label tensor.



In [ ]:
seed_everything()

total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)
train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
val_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train, num_val+num_train)]
test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_val + num_train, total)]

train_text = [(texts[i], index[i],label[i],'null') for i in range(num_train)]
val_text = [(texts[i], index[i],label[i],'null') for i in range(num_train, num_val+num_train)]
test_text = [(texts[i], index[i],label[i],'null') for i in range(num_val + num_train, total)]

#df2 = pd.DataFrame(train_text, columns=['text', 'seriel','label_id','lang'])
#df3 = pd.DataFrame(val_text, columns=['text', 'seriel', 'label_id','lang'])
#df4 = pd.DataFrame(test_text, columns=['text', 'seriel', 'label_id','lang'])
#df3 = df3.append(df2)

#print(df3.text.values)
#df3.to_csv('train_data.tsv', sep='\t')
#df4.to_csv('test_data.tsv', sep='\t')

In [ ]:
seed_everything()

total_a = len(df_a)

num_train_a = int(total_a * 0.9)
num_val_a = total_a - num_train_a

# make lists of 3-tuples (already shuffled the dataframe in a cell above)
train_set_a = [(input_ids_a[i], attention_masks_a[i], labels_a[i]) for i in range(num_train_a)]
val_set_a = [(input_ids_a[i], attention_masks_a[i], labels_a[i]) for i in range(num_train_a, total_a)]

train_text_a = [(texts_a[i], index_a[i], label_a[i]) for i in range(num_train_a)]
val_text_a = [(texts_a[i], index_a[i], label_a[i]) for i in range(num_train_a, total_a)]

#df2 = pd.DataFrame(train_text, columns=['text', 'seriel', 'label_id','labels'])
#df3 = pd.DataFrame(val_text, columns=['text', 'seriel', 'label_id','labesl'])
#df4 = pd.DataFrame(test_text, columns=['text', 'seriel', 'label_id'])
#print(df2.text.values,df2.labels.values)
#print(df3.text.values)
#print(df4.text.values)


# Save the train and val datasets to TSV files
#df3.to_csv('train_data.tsv', sep='\t', index=False)


["Immigration policies must align with our constitutional principles, safeguarding rights for all. Let's advocate for fairness and legality."
 'Victories in same-sex marriage encourage LGBTQ+ youth by providing positive role models.'
 'The increase in individuals seeking asylum has incited a contentious discourse at the national level about the issue of border security.'
 'Gay marriage is good for the economy, Czech companies say'
 'Public sentiment on immigration varies widely, reflecting diverse perspectives on economic impact, cultural identity, and national security.'
 'Victories contribute to a positive shift in societal norms, promoting acceptance and understanding.'
 'And despite the overall professional, educational, and financial success many Indian Americans enjoy, this has not inoculated them from the forces of discrimination, polarization, and contestation over questions of belonging and identity.'
 'Political Advancements in Immigration Reform: An International Outlook.'
 

Here we choose the model we want to finetune from https://huggingface.co/transformers/pretrained_models.html. Because the task requires us to label sentences, we wil be using BertForSequenceClassification below. You may see a warning that states that `some weights of the model checkpoint at [model name] were not used when initializing. . .` This warning is expected and means that you should fine-tune your pre-trained model before using it on your downstream task. See [here](https://github.com/huggingface/transformers/issues/5421#issuecomment-652582854) for more info.

In [ ]:
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification, BertConfig
from torch.optim import AdamW

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", # Use the 12-layer English BERT model, with an uncased vocab.
    num_labels = 15, # The number of output labels.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

# TODO: ACTION REQUIRED #

Define your fine-tuning hyperparameters in the cell below (we have randomly picked some values to start with). We want you to experiment with different configurations to find the one that works best (i.e., highest accuracy) on your validation set. Feel free to also change pretrained models to others available in the HuggingFace library (you'll have to modify the cell above to do this). You might find papers on BERT fine-tuning stability (e.g., [Mosbach et al., ICLR 2021](https://openreview.net/pdf?id=nzpLWnVAyah)) to be of interest.

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Set the seed value all over the place to make this reproducible.
seed_everything()

learning_rate = 2e-5
epsilon = 1e-8

weight_decay = 0.01

# Batch size
batch_size = 50

# Number of training epochs
epochs = 10

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=learning_rate,
    eps=epsilon,
    weight_decay=weight_decay
)

# Set up the learning rate scheduler with warmup
num_warmup_steps = 0.1 * len(train_set) / batch_size  # 10% of total steps
num_training_steps = len(train_set) / batch_size * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Fine-tune your model
Here we provide code for fine-tuning your model, monitoring the loss, and checking your validation accuracy. Rerun both of the below cells when you change your hyperparameters above.

In [ ]:
# function to get validation accuracy
def get_validation_performance(val_set,tokenizer):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1

    total_correct = 0
    incorrect_predictions = []
    for i in range(num_batches):

      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]

      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)

      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():

        # Forward pass, calculate logit predictions.
        # Note: this line of code might need to change depending on the model
        # the current line will work for bert-base-uncased
        # please refer to huggingface documentation for other models
        outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
        #outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)


        loss = outputs.loss
        logits = outputs.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = (logits).detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()


        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = np.argmax(label_ids, axis=1).flatten()

        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct

    # Keep track of incorrectly predicted sentences
        incorrect_mask = pred_flat != labels_flat
        incorrect_indices = np.where(incorrect_mask)[0]

        for idx in incorrect_indices:
            incorrect_predictions.append({
                'input_text': tokenizer.decode(input_id_tensors[idx].cpu().numpy(), skip_special_tokens=True),
                'predicted_label': pred_flat[idx],
                'true_label': labels_flat[idx]
            })

    # Report the final accuracy for this validation run.
    print("Num of correct predictions =", total_correct)
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy, incorrect_predictions



In [ ]:
import random
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

seed_everything()

best_val_acc = 0.0
best_model_path = "/content/best_model.pth"

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    num_batches = int(len(train_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(train_set))

      batch = train_set[i*batch_size:end_index]

      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)

      optimizer.zero_grad()

      # Perform a forward pass (evaluate the model on this training batch).
      # this line of code might need to change depending on the model
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()

      # Update the learning rate
      scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / num_batches

    current_learning_rate = optimizer.param_groups[0]['lr']
    print(f"Current Learning Rate: {current_learning_rate:.8f}")

    print("")
    print("Average training loss: {0:.2f}".format(avg_train_loss))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc, inc_pred = get_validation_performance(val_set, tokenizer)
    print(f"Validation accuracy: {val_acc}")
    #I = []
    #for i in range(5):
    #    I.append(inc_pred[i]['input_text'])
    #print(I)

    # TODO: SAVE YOUR MODEL HERE... (Refer PyTorch documentation for how to save models)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved the best model with validation accuracy: {best_val_acc}")

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
Current Learning Rate: 0.00001809

Average training loss: 0.60
Total loss: 6.036764160082734
Num of correct predictions = 4
Validation accuracy: 0.06779661016949153
Saved the best model with validation accuracy: 0.06779661016949153

======== Epoch 2 / 10 ========
Training...
Current Learning Rate: 0.00001598

Average training loss: 0.44
Total loss: 4.428893435584647
Num of correct predictions = 2
Validation accuracy: 0.03389830508474576

======== Epoch 3 / 10 ========
Training...
Current Learning Rate: 0.00001386

Average training loss: 0.38
Total loss: 3.7566808925781934
Num of correct predictions = 10
Validation accuracy: 0.1694915254237288
Saved the best model with validation accuracy: 0.1694915254237288

======== Epoch 4 / 10 ========
Training...
Current Learning Rate: 0.00001175

Average training loss: 0.34
Total loss: 3.3994094471917267
Num of correct predictions = 10
Validation accuracy: 0.1694915254237288

======== Epoch 5 / 10 ======

# Evaluate your model on the test set
After you're satisfied with your hyperparameters (i.e., you're unable to achieve higher validation accuracy by modifying them further), it's time to evaluate your model on the test set! Run the below cell to compute test set accuracy.


In [ ]:
seed_everything()

# If your notebook disconnects during training, then here, first load the best
# model you saved (refer PyTorch docs), then check validation performance

val_acc_test, inc_pred_test = get_validation_performance(test_set, tokenizer)
print(f"Validation accuracy: {val_acc_test}")


Num of correct predictions = 9
Validation accuracy: 0.14754098360655737


## Question 8 (10 points):
Finally, perform an *error analysis* on your model. This is good practice for your final project. Write some code in the below code cell to print out the text of up to five test set examples that your model gets **wrong**. If your model gets more than five test examples wrong, randomly choose five of them to analyze. If your model gets fewer than five examples wrong, please design five test examples that fool your model (i.e., *adversarial examples*). Then, in the following text cell, perform a qualitative analysis of these examples. See if you can figure out any reasons for errors that you observe, or if you have any informed guesses (e.g., common linguistic properties of these particular examples). Does this analysis suggest any possible future steps to improve your classifier?

In [ ]:
seed_everything()
torch.cuda.empty_cache()

## YOUR ERROR ANALYSIS CODE HERE
## print out up to 5 test set examples (or adversarial examples) that your model gets wrong
I = []
for i in range(5):
    I.append(inc_pred[i])
for i in I:
    print(i)

{'input_text': 'advocates for health stress the importance of healthcare policies inclusive of lgbtq + individuals', 'predicted_label': 0, 'true_label': 8}
{'input_text': "the migration process implies complex challenges in terms of governance, migrant workers'protection, migration and development linkages, and international cooperation.", 'predicted_label': 0, 'true_label': 5}
{'input_text': 'the media has a key role in creating popular perceptions of immigration, exerting influence over public opinion and policy debates.', 'predicted_label': 0, 'true_label': 11}
{'input_text': 'the contributions made by second - generation immigrants, who are born in a host nation to parents who are immigrants, play a significant role in shaping the ongoing discourse surrounding cultural identity.', 'predicted_label': 0, 'true_label': 10}
{'input_text': 'the legal frameworks pertaining to same - sex marriage are through a process of evolution, as legislators strive to address concerns of prejudice an

### *DESCRIBE YOUR QUALITATIVE ANALYSIS OF THE ABOVE EXAMPLES IN YOUR REPORT*



---

Finished? Remember to upload the PDF file of this notebook, report and your three dataset files (annotator1.tsv, annotator2.tsv, and final_data.tsv) to Gradescope with the filename line formatted as **Firstname_Lastname_HW2**.
